# Рекомендация тарифов

В вашем распоряжении данные о поведении пользователей тарифов. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

Постройте модель с максимально большим значением *accuracy*. Нужно довести долю правильных ответов по крайней мере до 0.75.

<b> Этапы задачи </b>
1. Импорт необходимых библиотек и модулей
2. Открытие файла и первичный взгляд на данные
3. Разбивка данные на тренировочную и тестовую выборки.
4. Проверка модели на адекватности с помощью Dummy
5. Проверка модели "решающего дерева"
6. Проверка модели "случайного леса"
7. Проверка модели логистической регрессии
7. Тестирование всех моделей на тестовой выборке. Сравнение показетелей.
8. Выбор оптимальной модели и общий вывод.

## Откройте и изучите файл

### Импорт библиотек и модулей

In [93]:
import time

#импортируем библиотеки
import pandas as pd

#импортируем встроенные ф-ции sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

#импортируем алгоритмы обучения модели
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier as dummy_clf

### Первичный взгляд на данные

In [94]:
data = pd.read_csv('/datasets/users_behavior.csv')
print('Количество пропусков в датасете:', data.isna().sum())
print(data.shape)
data.head(5)

Количество пропусков в датасете: calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64
(3214, 5)


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<b>ВЫВОД:</b> 3214 наблюдений в датасете, 4 признака и один целевой признак. Типы данные - floatы. Пропусков нет, предобработка не требуется. 

## Разбейте данные на выборки

In [95]:
# отделим фичи от таргетов
features = data.drop(['is_ultra'], axis = 1)
target = data['is_ultra']

In [96]:
# разобьем данные на выборки
features_train, features_test, target_train, target_test = train_test_split(features, target,
                                                                             test_size = 0.2, random_state = 42)

#проверим соотношение выборок
print('Размерность обучающей выборки:', len(features_train), 
      'Размерность тестовой выборки:',len(features_test), sep = '\n')

Размерность обучающей выборки:
2571
Размерность тестовой выборки:
643


## Исследуйте модели

### Проверка модели на адекватность с помощью Dummy

In [105]:
model_dummy = dummy_clf(strategy = 'most_frequent')
model_dummy.fit(features_train,target_train)
print('Accuracy случайной модели на тестовой выборке составила',model_dummy.score(features_test, target_test))

Accuracy случайной модели на тестовой выборке составила 0.7076205287713841


<b>ВЫВОД:</b> показатели случайной модели довольно высокие. Используем это как стартовую точку, выше которой должно быть значение accuracy.

In [98]:
# создадим пустой список для сохранения результатов предсказаний
results = []
models = ['Decisio Tree', 'Random Forest', 'Logistic Regression']

### Исследование модели решающего дерева. 
При обучении мы будем tune-ть:
- criterion (gini vs entropy)
- random_state - зададим значение 42 для обеспечения постоянства результатов
- min_samples_leaf - возьмем значения от 1 до 50

In [99]:
# создадим сетку параметров
params = {'max_depth' : [5, 10, 15],
         'criterion' : ['gini', 'entropy'],
         'min_samples_leaf' : [1, 5, 9, 20, 30, 40]}

In [100]:
%time
# найдем лучшую модель поиском по сетке
tree_clf = DecisionTreeClassifier(random_state = 42)
grid_model = GridSearchCV(estimator = tree_clf,
                         param_grid  = params,
                         scoring = 'accuracy',
                         verbose = 0,
                         n_jobs = -1,
                         cv = 3)
grid_model.fit(features_train, target_train)
print(grid_model.best_estimator_)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs
DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_leaf=20,
                       random_state=42)


In [101]:
# объявим лучшую модель дерева решений
best_tree = grid_model.best_estimator_

# проверим модель на тестовой выборке
predictions = best_tree.predict(features_test)
result = accuracy_score(pd.Series(predictions), target_test)
results.append(result)
print('Лучший показатель accuracy дерева решений равен', result)

Лучший показатель accuracy дерева решений равен 0.8118195956454122


### Исследование модели случайного леса

При исследовании будем тестировать 4 параметра:
- критерий (джини vs энтропия)
- msx_depth
- min_samples_leaf (минимальное число наблюдений в терминальном узле)
- n_estimators (число оценщиков)

In [102]:
# создадим сетку параметров
params = {'max_depth' : [5, 10, 15],
         'criterion' : ['gini', 'entropy'],
         'min_samples_leaf' : [1, 5, 9, 20, 30, 40],
         'n_estimators' : [5,10,15,20,30]}

In [103]:
%time
# найдем лучшую модель поиском по сетке
forest_clf = RandomForestClassifier(random_state = 42)
grid_model = GridSearchCV(estimator = forest_clf,
                         param_grid  = params,
                         scoring = 'accuracy',
                         verbose = 0,
                         n_jobs = -1,
                         cv = 3)
grid_model.fit(features_train, target_train)
print(grid_model.best_estimator_)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs
RandomForestClassifier(max_depth=10, n_estimators=30, random_state=42)


In [104]:
# сохраним лучшую модель
best_forest = grid_model.best_estimator_

# проверим модель на тестовой выборке
predictions = best_forest.predict(features_test)
result = accuracy_score(pd.Series(predictions), target_test)
results.append(result)
print('Лучший показатель accuracy случайного леса равен', result)

Лучший показатель accuracy случайного леса равен 0.8195956454121306


### Изучение модели логистической регрессии

In [120]:
model = LogisticRegression(random_state = 42, solver = 'liblinear')
model.fit(features_train, target_train) 
predictions = model.predict(features_test) 
result = accuracy_score(target_test, predictions)
results.append(result)
    
print('Наилучший показатель accuracy логистической регрессии:', result)

Наилучший показатель accuracy логистической регрессии: 0.7200622083981337


## Проверьте модель на тестовой выборке

In [124]:
pd.DataFrame(index = models, columns = ['Accuracy Score'], data = results)

,Accuracy Score
Decisio Tree,0.811820
Random Forest,0.819596
Logistic Regression,0.720062


<b>ВЫВОД:</b> на тестовой выборке лучше всего показала себя модель случайного леса, дерево решений - на втором месте, хуже всего сработала линейная регрессия.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
